In [2]:
import pandas as pd
import ast
import re

# Set the display options to show all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

# Data import Aisha
original_dataframe = pd.read_csv('recipes_w_search_terms.csv')

# data import katie
#original_dataframe = pd.read_csv("C:\\Users\\k.ritscher\\Desktop\\Bootcamp\\Final_Project\\archive\\recipes_w_search_terms.csv")

In [3]:
# drop duplicates
df = original_dataframe.copy()
df.drop_duplicates(subset=["name", "ingredients", "description"], keep='first', inplace=True)
   

In [4]:
# see difference in the size of the df,
print(len(original_dataframe))
print(len(df))

494963
494776


In [5]:
# creating a smaller test-dataframe
test_df = df.iloc[:10, :]

In [6]:
def filter_basic_ingredients(ingredient_list):
    basic_ingredients = ['salt', 'oil', 'water']  # Removed the extra double quote in 'salt'

    return [
        ingredient for ingredient in ingredient_list if ingredient.lower() in basic_ingredients
    ]  # Use lower() to perform case-insensitive comparison


test_df['filtered_ingredients'] = test_df['ingredients'].apply(filter_basic_ingredients)

# Display the modified DataFrame
test_df


/var/folders/jl/tcgfk4mx3gg3jtw3rcbk08pw0000gp/T/ipykernel_52896/1873001710.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_df['filtered_ingredients'] = test_df['ingredients'].apply(filter_basic_ingredients)


,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,filtered_ingredients
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}",[]
1,232037,Simple Shrimp and Andouille Jambalaya,"Simple, easy and very tasty for when you are in the mood for jambalaya. My kids loved it! From Cook's Illustrated ""Quick Recipe"" Cookbook.","['onion', 'red bell pepper', 'garlic cloves', 'large shrimp', 'salt', 'hot pepper sauce', 'vegetable oil', 'andouille sausage', 'long grain rice', 'bay leaves', 'diced tomatoes', 'clam juice', 'fresh parsley']","[""1 medium onion, chopped coarse "",""1 medium red bell pepper, chopped coarse "",""5 medium garlic cloves, chopped coarse "",""1 lb extra large shrimp, shelled and deveined "","" salt"","" hot pepper sauce"",""1 tablespoon vegetable oil"",""3/4 lb andouille sausage, halved lengthwise and then cut into 1/4 inch slices "",""1 1/2 cups long grain rice"",""4 bay leaves"",""1 (14 ounce) can diced tomatoes, briefly drained "",""2 (8 ounce) bottles clam juice"",""1/4 cup fresh parsley, chopped ""]",1 (366 g),4,"['In a food processor, pulse the onion, red pepper and garlic until chopped very fine, ten to twelve 1-second pulses, scraping down the sides of the bowl as necessary; set aside.', 'Season the shrimp with salt and hot pepper sauce to taste.', 'Heat the oil in a large Dutch oven over medium high heat until shimmering. Add the shrimp in a single layer and cook, without stirring, for 30 seconds. Using tongs, flip the shrimp and cook for another 30 seconds. Transfer the shrimp to a medium bowl and set aside.', 'Add the sausage to the pan and cook, stirring occasionally, until lightly browned, about 3 minutes. Using a slotted spoon, transfer the sausage to a second bowl.', 'Scrape the veggies from the food processor into the empty pot and cook, stirring frequently, until softened, about 3 minutes. Stir in the rice and continue to stir until the grains are coated with the fat, about 1 minute.', 'Add the bay leaves, tomatoes, clam juice, and 1 cup water and bring to a boil.', 'Stir in the sausage, cover, and reduce the heat to low.', 'Cook until the rice is tender, 17 to 20 minutes.', 'Off the heat, stir in the shrimp and parsley with a fork (do not mush the rice), cover the pot, and allow to sit for 2 minutes or until the shrimp is heated through.', 'Discard the bay leaves and adjust the seasoning with salt and h

In [7]:
# Function to extract 'time-to-make' value based on 'minutes' or 'hours'
def extract_time_to_make(tags_list):
    time_to_make_values = []
    for tag in tags_list:
        if 'minutes' in tag.lower() or 'hours' in tag.lower():
            time_to_make_values.append(tag.strip())
    return time_to_make_values if time_to_make_values else None

# Apply the function to create the new 'time-to-make' column
df['time-to-make'] = df['tags'].apply(lambda x: extract_time_to_make(ast.literal_eval(x)) if pd.notnull(x) else None)
df.head(1)

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,time-to-make
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}",[4-hours-or-less]


In [8]:
import re

# Function to extract 'low-in-something' value based on 'low' 
def extract_low_in_something(low_values_list):
    low_in_something_values = []
    for tag in low_values_list:
        # Check if the tag has 'low' as a whole word
        if re.search(r'\blow\b', tag.lower()):
            low_in_something_values.append(tag.strip())
    return low_in_something_values if low_in_something_values else None

# Apply the function to create the new 'low-in-something' column
df['low-in-something'] = df['tags'].apply(lambda x: extract_low_in_something(ast.literal_eval(x)) if pd.notnull(x) else None)
df.head(1)

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,time-to-make,low-in-something
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}",[4-hours-or-less],"[low-cholesterol, low-calorie, low-carb, low-in-something]"


In [9]:
import re

# Function to extract 'free-of-something' value based on 'free' 
def extract_free_of_something(free_values_list):
    free_of_something_values = []
    for tag in free_values_list:
        # Check if the tag has 'free' as a whole word
        if re.search(r'\bfree\b', tag.lower()):
            free_of_something_values.append(tag.strip())
    return free_of_something_values if free_of_something_values else None

# Apply the function to create the new 'free-of-something' column
df['free-of-something'] = df['tags'].apply(lambda x: extract_free_of_something(ast.literal_eval(x)) if pd.notnull(x) else None)
df.head(1)

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,time-to-make,low-in-something,free-of-something
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}",[4-hours-or-less],"[low-cholesterol, low-calorie, low-carb, low-in-something]",None


In [10]:
# List of predefined cuisines
cuisines = [
    'indian', 'american', 'lebanese', 'italian', 'chinese', 'mexican', 'japanese', 'french', 'german', 'greek', 'spanish',
    'thai', 'korean', 'vietnamese', 'brazilian', 'moroccan', 'turkish', 'russian', 'swedish', 'caribbean', 'ethiopian',
    'peruvian', 'malaysian', 'australian', 'south african', 'portuguese', 'cajun/creole', 'argentine', 'filipino',
    'middle-eastern', 'north-american', 'central-american', 'south-american', 'european', 'eastern-european',
    'western-european', 'southeast-asian', 'south-asian', 'middle-eastern', 'north-african', 'sub-saharan-african',
    'south-indian', 'north-indian', 'southern-american', 'northern-american', 'central-european', 'northern-european',
    'southern-european', 'eastern-asian', 'western-asian', 'northern-african', 'southern-african', 'east-african',
    'west-african', 'eastern-european', 'western-european', 'southeast-asian', 'south-asian', 'east-asian',
    'central-asian', 'western-asian', 'middle-eastern', 'south-east-asian', 'north-east-asian', 'south-central-asian',
    'north-central-asian', 'south-west-asian', 'north-west-asian', 'south-east-european', 'north-east-european',
    'south-central-european', 'north-central-european', 'south-west-european', 'north-west-european', 'south-east-african',
    'north-east-african', 'south-central-african', 'north-central-african', 'south-west-african', 'north-west-african',
    'southwest', 'southwestern-us', 'southwestern-american', 'southwestern', 'south-west-pacific'
]

# Function to extract 'cuisine' value
def extract_cuisines(tags_list):
    cuisine_list = [tag.strip() for tag in tags_list if tag.lower() in cuisines]
    return cuisine_list if cuisine_list else None

# Apply the function to create the new 'cuisine' column
df['cuisine'] = df['tags'].apply(lambda x: extract_cuisines(ast.literal_eval(x)) if pd.notnull(x) else None)

df.head(1)

,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,time-to-make,low-in-something,free-of-something,cuisine
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}",[4-hours-or-less],"[low-cholesterol, low-calorie, low-carb, low-in-something]",None,None


In [16]:
# Attempt to extract the main ingredients of a recipe based on logic that extracts words from the recipe name, then looks for those words in the ingredients. The words that are present in the name are considered the main ingredients of the recipe.

import pandas as pd
import re

# Assuming 'name' and 'ingredients' are columns in your DataFrame df
def extract_main_ingredients(row):
    recipe_name = row['name'].lower()

    # Extract words from ingredients and convert to lowercase
    ingredients_lower = [word.lower() for word in re.findall(r'\b\w+\b', ' '.join(row['ingredients']))]

    # Find all ingredients mentioned in the recipe name
    main_ingredients = [ingredient for ingredient in ingredients_lower if ingredient in recipe_name]

    return main_ingredients

# Apply the extract_main_ingredients function to create a new column 'main_ingredients'
df['main_ingredients'] = df.apply(extract_main_ingredients, axis=1)

# Display the modified DataFrame
df.head(1)


,id,name,description,ingredients,ingredients_raw_str,serving_size,servings,steps,tags,search_terms,time-to-make,low-in-something,free-of-something,cuisine,main_ingredients
0,96313,Grilled Garlic Cheese Grits,"We love grits, this is another good way to serve them. A great alternative to a baked potato when served with grilled steak or chicken. I belive this recipe could be made with instant grits.The 2 1/2 hours for refrigeration is not include in time. The recipe comes from Tast of Home's Light and Tasty.","['water', 'grits', 'salt', 'cheddar cheese', 'garlic', 'olive oil']","[""4 cups water"",""1 cup uncooked old fashion grits"",""1 teaspoon salt"",""4 ounces shredded cheddar cheese"",""1 -2 clove garlic, minced "",""1 tablespoon olive oil""]",1 (155 g),8,"['I a sauce pan, bring water to a boil; slowly add grits and salt, stirring constantly; Reduce heat:simmer, uncovered, for 40-45 minutes or untill thickened, stirrin occasionally.', 'Add cheese and garlic; stir until cheese is melted, Spray 9-inch baking dish with nonstick cooking spray; Cover and refrigerate for 2 to 2 1/2 hours or until frim.', 'Before starting the grill, coat the grill rack with nonstick cooking spray; Cut the grits into 3-inch squares; Brush both sides with olive oil.', 'Grill, covered, over medium heat for 4 to 6 minutes on each side or until lightly browned.']","['time-to-make', 'course', 'main-ingredient', 'preparation', 'occasion', 'side-dishes', 'eggs-dairy', 'refrigerator', 'diabetic', 'vegetarian', 'grains', 'cheese', 'stove-top', 'dietary', 'low-cholesterol', 'low-calorie', 'comfort-food', 'low-carb', 'low-in-something', 'pasta-rice-and-grains', 'brunch', 'taste-mood', 'equipment', 'presentation', 'served-hot', '4-hours-or-less']","{'diabetic', 'low-calorie', 'vegetarian', 'low-carb', 'side'}",[4-hours-or-less],"[low-cholesterol, low-calorie, low-carb, low-in-something]",None,None,"[a, t, e, r, g, r, i, t, s, s, a, l, t, c, h, e, d, d, a, r, c, h, e, e, s, e, g, a, r, l, i, c, l, i, e, i, l]"
